In [ ]:
# Autor: Daniel Pinto
# Introduccion a la notacion asintotica parte 2
# Fecha: 2021/09/23 YYYY/MM/DD
from typing import List, TypeVar, Tuple, Any, Callable
from hypothesis import given, strategies as st
from IPython.display import Markdown, display
from itertools import accumulate
from functools import reduce

def display_(s : str) -> None:
    '''
    A way to display strings with markdown 
    in jupyter.
    '''
    display(
        Markdown(s)
    )


SUCCESS_COLOR = '#4BB543'
ERROR_COLOR   = '#B00020'

def color_text(s : str, color : str =SUCCESS_COLOR ) -> str:
    return f"<span style='color:{color}'> {s} </span>."


a      = TypeVar('a')
b      = TypeVar('b')
c      = TypeVar('c')

# Recursividad, El teorema maestro y Divide and Conquer

El principio de Divide and Conquer es bastante elegante: Dado un problema de tamaño $n$, lo podemos dividir en $a$ subproblemas de tamaño $\frac{n}{b}$ cada uno, y luego combinamos las soluciones usando algun otro algoritmo con complejidad $f(n)$.


Esto es mejor ejemplificado usando un ejercicio:

# Ejercicio: Maximum Subarray Sum

Dado un arreglo, dar el sub arreglo cuya suma es maxima.

```
Input: arr = [−2, 1, −3, 4, −1, 2, 1, −5, 4]
Output: (3,6)
Output:  puesto que: arr[3] + arr[4] + arr[5] + arr[6] = 6 y es de suma maxima.
```

In [ ]:

# [a,mid,b]
# [a]
# [a,mid]   pertenece exclusivamente a la parte izquierda
# [mid+1,b] pertenece exclusivamente a la parte derecha
# pasa por la mitad, a1,b1: [a..a1,mid,b1..b]
def _max_subarray_sum(arr : List[float], interval : Tuple[int,int]) -> Tuple[float,int,int] :
    (a,b) = interval
    # Lista de un solo elemento o vacia
    if a>=b:
        try:
            return (arr[a],a,a)
        except:
            return (0,-1,-1)
    # Lista de dos elementos [a,b]
    if a+1==b:
        # se retorna el maximo entre:
        # a+b si ambos son positivos
        # a   si b es mas negativo que a
        # b   si a es mas negativo que b 
        return max(
            [(arr[a] + arr[b],a,b)
            , (arr[a],a,a)
            , (arr[b],b,b)
            ]
            )

    # Sacamos el punto medio usando division entera
    mid : int = (a+b) // 2
    # Paso recursivo: obtenemos el valor maximo del arreglo
    # izquierdo y derecho que no atraviesan el centro
    (maxL,maxL_a,maxL_b) = _max_subarray_sum(arr,(a,mid))
    (maxR,maxR_a,maxR_b) = _max_subarray_sum(arr,(mid,b))

    # Para los que atraviesan el centro, usaremos esta funcion:
    def get_max_val_interval(sub : List[float]) -> Tuple[float,int]:
        # si [1,2,-5,4] es el arreglo del centro retorna:
        # max : [(0,0),(1,1),(3,2),(-2,3),(2,4)] = (3,2)
        # es decir, la suma maxima es 3 y empieza en el indice 2
        return max(
            accumulate(
                sub,
                func=lambda x_i,val: (x_i[0]+val,x_i[1]+1),
                initial=(0,0)
            )
        )

        # 
        # foldl (+) 0 [1..3] = [0,1,3,5]
        # scanl (+) 0 [1..3] = [0,1,3,5]
    
    # Obtenemos la suma maxima e indice del arreglo que esta a la derecha
    # de mid (sin incluirlo)
    (maxMR,_maxR_b) = get_max_val_interval(arr[mid+1:b+1])
    # Obtenemos el arreglo que esta a la izquierda de mid (sin incluirlo)
    # necesitamos esto puesto que si a=0, entonces arr[mid:a-1:-1] = arr[mid:-1:-1] = []
    # y nosotros queremos obtener la lista de la izquiera, no una no vacia
    aux : List[float] = arr[mid-1:a-1:-1] if a != 0 else arr[mid-1::-1]
    # Obtenemos la suma maxima e indice del arreglo que esta a la derecha
    # de mid (sin incluirlo)
    (maxML,_maxL_a) = get_max_val_interval(aux)
    # fijemonos que tanto maxM_b como maxM_a (los indices), estan con respecto al arreglo interno:
    # es decir, si nuestro arreglo original es: [2,[1,2,5,4,7],5]
    # entonces: (maxMR,maxMR_b) = (11,1) porque 4+7=11 
    # por lo tanto necesitamos desplazar el indice a la posicion: mid+indice en el caso
    # de la derecha y mid-indice en el caso de la izquierda
    maxM_b         = _maxR_b + mid
    maxM_a         = mid - _maxL_a 
    # el maximo valor que pasa por el centro
    # es la suma del maximo valor de derecha + izquierda + la pieza del centro
    maxM           = maxMR + maxML + arr[mid]
    
    # finalmente, retornamos  el valor maximo
    if maxM == max([maxL,maxR,maxM]):
        return (maxM,maxM_a, maxM_b)
    if maxL == max([maxL,maxR,maxM]):
        return (maxL,maxL_a,maxL_b)
    
    return (maxR,maxR_a,maxR_b)


def max_subarray_sum(arr : List[float]) -> Tuple[float,int,int]:
    return _max_subarray_sum(arr,(0,len(arr)-1))

def sum_array_test(f : Callable[[List[float]],Tuple[float,int,int]]) -> None:
    @given(ys = st.lists(st.integers(min_value=-20,max_value=20),max_size=20),)
    def _sum_array_test(ys : Any) -> None:
        xs : List[float] = list(ys)
        n : int = len(xs)
        indexes : List[Tuple[int,int]] = [(x,y) for x in range(n) for y in range(x,n)]

        try:
            (maximum_sum,index) = max(map(lambda a_b: (sum(xs[a_b[0]:a_b[1]+1]),a_b), indexes))
        except:
            (maximum_sum,index) = (0,(-1,-1))

        (val,i,j) = f(xs)

        tol = 10**(-10)

        try:
            assert(abs(val-maximum_sum) < tol)
        except Exception:
            display_(
                f"{color_text('Test Failed!',color=ERROR_COLOR)}<br>" +
                f"The maximum sum was: `{maximum_sum}` <br>" + 
                f"Got: `{val}` <br>"  +
                f"Array: <br>" +
                f"`{xs}`<br>" + 
                f"Solution indexes: `{index}` + <br>" + 
                f"Got: `{(i,j)}`"
            )
            return
        
        display_(color_text("Test successful!"))

    _sum_array_test()


arr : List[float] = [-2, 1, -3, 4, -1, 2, 1, -5, 4]
(val,a,b)         = max_subarray_sum(arr)
print((val,a,b))
sum_array_test(max_subarray_sum)



Fijemonos que el algoritmo general el siguiente arbol de recursion:

<center>

![max subarray sum recursion tree](./Images/arbol_recursivo.png)

</center>

Cada nodo tiene exactamente $a=2$ hijos, los cuales trabajan con $\frac{n}{b=2}$ de los datos a la vez, ademas, notemos que para combinar los resultados, hacemos: $f=O(n)$ operaciones: encontrar el subarreglo maximo de $[a,mid)$, encontrar el subarreglo maximo $(mid,b]$, para finalmente unirlos $[a,b]$ y retornar el maximo entre este y los resultados recursivos.

Esto sugiere que debemos recorrer todo el arbol, y que el costo de traslado de cada nodo es $O\left(\dfrac{N}{2^{nivel}} \right)$, por lo tanto, nuestra complejidad sera:

$$
O\left(\sum_{0 \leq i \leq log_2(N)}  (a=2^i) \cdot \dfrac{f=N}{b=2^i} \right) = O\left(\sum_{0 \leq i \leq log_2(N)} N \right) = O(N \ log(n))
$$

# Teorema maestro

<center>

![teorema maestro](./Images/teorema_maestro.png)

</center>


En el caso anterior, notemos que:


$$
f(n) = O(n) = O(n \cdot 1) = O(n \cdot log^0(n)) \implies k = 0
$$

Entonces, por el caso 2 del teorema maestro, la complejidad sera:

$$
O(n^{log_2(2)} \cdot log^{0+1}n) = O(n \ log(n))
$$


# Ejercicio: Revisitando Max sum array sum: Kadane's Algorithm

Notemos que el problema del sub arreglo maximo lo podemos simplificar aun mas, sea `arr = [1,2,-3,-4,1,10]` entonces, trabajaremos recorriendo el arreglo:

- `i=0` inicialmente el arreglo esta compuesto de solo 1, entonces nuestra suma maxima por ahora sera: `max_sum=1`, y nuestra suma acumulada: `acc_sum=1`
- `i=1` luego descubrimos el 2, por lo tanto aumentamos nuestra suma acumulada: `acc_sum += 2 = 3`, y como descubrimos un nuevo maximo, lo actualizamos: `max_sum=3`
- `i=2` Luego descubrimos al -3, aumentando nuestra suma acumulada: `acc_sum += -3 = 0`, esto no es un nuevo maximo asi que proseguimos
- `i=3`, descubrimos -4, lo que hace que nuestra suma acumulada sea: `acc_sum=-4`, fijemonos que ahora la suma es negativa, eso quiere decir que no importa cual sea el numero que le prosigue,
  si le sumamos lo que llevamos acumulado siempre va a decrementar su valor, por lo tanto lo mejor que podemos hacer es resetear `acc_sum`, simbolizando que si hay una nueva mejor suma, esta va a estar a la derecha de la vieja
- `i=4`, descubrimos 1, actualizando: `acc_sum=1`
- `i=5`, descubrimos 10, lo que convierte: `acc_sum=11`, y como es mayor que nuestra mejor suma actual, la actualizamos: `max_sum=11`

Claramente, la complejidad de este algoritmo es lineal, veamos una posible implementacion:


In [ ]:
def _kadane(arr: List[float]) -> Tuple[float,int,int]:
    if arr == []:
        return (0,-1,-1)

    acc_sum : float = 0
    max_sum : float = 0
    n       : int   = len(arr)
    a       : int   = 0
    b       : int   = 0


    for i in range(n):
        acc_sum += arr[i]
        if acc_sum > max_sum:
            max_sum = acc_sum
            b       = i
        if acc_sum < 0:
            acc_sum = 0
            a       = i

    
    if all([x < 0 for x in arr]):
        return max([(x,i,i) for (x,i) in zip(arr,range(n))]) 

    return (max_sum,a,b)




sum_array_test(_kadane)
#print(_kadane([-1,-5,-3,-2]) )

# Ejercicio: Binary Search

Dado un arreglo ordenado `arr`, y un elemento `n`, decidir si `n` esta en el arreglo:

```
Input: arr = [-10,-5,0,3,8,10,20,25], n = 20
Output: (True,2)
Output:  puesto que: arr[2]=0

Input: arr = [-10,-5,0,3,8], n = 50
Output: (False,-1)
Output:  puesto que 50 no pertenece a arr
```

In [ ]:

def bin_search(arr : List[int], n : int) -> Tuple[bool,int]:
    if arr==[]:
        return (False,-1)
    b : int = len(arr) - 1
    a : int = 0
    while(True):
        mid : int = (a+b) // 2
        if arr[mid] == n:
            return (True,mid)
        if a == b:
            return (False,-1)
        if a +1 == b:
            return (True,arr[a:b+1].index(n)+a) if n in arr[a:b+1] else (False,-1)
        if n > arr[mid]:
            a=mid
        if n < arr[mid]:
            b = mid
        

def rec_bin_search(arr : List[int], n : int) -> Tuple[bool,int]:

    def _rec_bin_search(a : int, b: int) -> Tuple[bool,int]:
        mid = (a+b) // 2
        
        if arr[mid] == n:
            return (True,mid)
        if a == b:
            return (False,-1)
        if a +1 == b:
            return (True,arr[a:b+1].index(n)+a) if n in arr[a:b+1] else (False,-1)

        if n > arr[mid]:
            return _rec_bin_search(mid, b)
        if n < arr[mid]:
            return _rec_bin_search(a, mid)
        
        return (False,-2)

    return  _rec_bin_search(0,len(arr)-1)


def _bin_search(arr : List[int], n : int) -> Tuple[bool,int]:
    def _bin_search_(a : int, b: int) -> Tuple[bool,int]:
        if arr[b]  < n:
            return (False,-1)
        if arr == []:
            return (False,-1)
        if a==b:
            if arr[a] == n:
                return (True,a)
            else:
                return (False,-1)
        if a+1 ==b:
            #[a,b]
            if arr[a] == n:
                return (True,a)
            if arr[b] == n:
                return (True,b)
            return  (False,-1)
        mid = (a+b) // 2

        if arr[mid]>=n:
            return _bin_search_(a,mid)
        else:
            return _bin_search_(mid,b)
            
    return _bin_search_(0,len(arr)-1)

def search(f : Callable[[List[int],int],Tuple[bool,int]]) -> None:

    @given(ys=st.lists(st.integers(min_value=-20,max_value=20),min_size=1, max_size=20 )  )
    def _search(ys : Any):
        xs : List[int] = list(ys)
        xs.sort()
        min_x = xs[0]
        max_x = xs[-1]
        n : int = len(xs)

        for i in range(min_x,max_x+1):
            sol = f(xs,i)
            if i in xs:
                inds = [j for j in range(n) if xs[j] == i]
                try:
                    assert(sol[0] and sol[1] in inds)
                except Exception:
                    display_(
                f"{color_text('Test Failed!',color=ERROR_COLOR)}<br>" +
                f"Expected: `{(True,inds)}` <br>" + 
                f"Got: `{sol}` <br>"  +
                f"Array: <br>" +
                f"`{xs}` <br>" + 
                f"Element: `{i}`"
                )
                    
            else:
                try:
                    assert(sol == (False,-1))
                except Exception:
                    display_(
                f"{color_text('Test Failed!',color=ERROR_COLOR)}<br>" +
                f"Expected: `{(False,-1)}` <br>" + 
                f"Got: `{sol}` <br>"  +
                f"Array: <br>" +
                f"`{xs}` <br>" + 
                f"Element: `{i}`"
                )
        display_(color_text("Test successful!"))
    
    _search()


#search(bin_search)
#search(rec_bin_search)
search(_bin_search)
#print(_bin_search([1,3,5,7,9,11,15],16))



# Complejidad:

Notemos que la formula recursiva es:

$$
T(n) = T\left(\dfrac{n}{2} \right) + O(1)
$$

Por lo tanto, por el teorema maestro, como $f(n) = 1$, y $O(1) = O(1 \cdot 1) = O(n^0 \cdot log^0(n))$, tenemos que la complejidad es:

$$
O(n^0 log(n)) = O(log(n))
$$


# Ejercicio: Encontrar el infimo de un numero en un arreglo ordenado

Sea $arr$ un arreglo ordenado, y $n$ cualquier numero, decimos que $inf$ es el infimo de $n$ en $arr$ si y solo si:

1. $inf \in arr$
2. Para cualquier $a \in arr$, si $a\leq n$ entonces $a \leq inf$
3. Para cualquier $b \in arr$, si $b \geq n$ entonces $b \geq inf$

Por ejemplo, si $arr=[1,5,10,15]$ y $n=7$, entonces $inf=5$ puesto que $1 \leq 5 \leq 7$
y $5 \leq 7 \leq 10 \leq 15$. 

Es decir, un infimo es la **mayor** de las cotas **inferioes**

Encontrar la posicion y el valor del infimo para cualquier lista **no vacia**.

In [ ]:
def get_inf(arr : List[int], n : int) -> Tuple[int,int]:

    def _get_inf(a : int, b: int) -> Tuple[int,int]:
        mid = (a+b) // 2

        if arr[mid] == n:
            return (n,mid)
        if (a == b) or (a +1 == b):
            return (arr[a],a)

        if n > arr[mid]:
            return _get_inf(mid, b)
        if n < arr[mid]:
            return _get_inf(a, mid)
        
        return (-1,-1)

    return  _get_inf(0,len(arr))

def test_inf(f : Callable[[List[int],int],Tuple[int,int]]) -> None:

    @given(ys=st.lists(st.integers(), min_size=2))
    def _test_int(ys : Any) -> None:
        from random import randrange
        def implies(p : bool, q: bool):
            return (not p) or q

        
        xs : List[int] = list(ys)
        xs.sort()
        aux : int      = randrange(1,max(2,len(xs)-1))
        n  : int       = xs[aux]
        xs.__delitem__(aux)
        
        
        def isInf(x : int) -> bool :
            less_cond = all([implies(a<=n,a<=x) for a in xs ] )
            grt_cond  = all([implies(a>=n,a>=x) for a in xs] )
            return less_cond and grt_cond and x<=n

        inf : int = [x for x in xs if isInf(x)][0]
        inds : List[int] = [j for j in range(len(xs)) if xs[j] == inf]
        (sol_inf,index)  = f(xs,n)
        try:
            assert(inf == sol_inf and index in inds)
        except Exception:
            display_(
                f"{color_text('Test Failed!',color=ERROR_COLOR)}<br>" +
                f"Expected: `{(inf,inds)}` <br>" + 
                f"Got: `{(sol_inf,index)}` <br>"  +
                f"Array: <br>" +
                f"`{xs}` <br>" + 
                f"n: `{n}`"
            )
        display_(color_text("Test successful!"))

    
    _test_int()


def _inf_search(arr : List[int], n : int) -> Tuple[int,int]:
    
    def _bin_search_(a : int, b: int) -> Tuple[int,int]:
        if a==b:
            #assert(arr[a] <= n)
            return (arr[a] ,a)
        if a+1 ==b:
            #[a,b]
            if arr[b] <= n:
                #assert(arr[b] <= n)
                return (arr[b],b)
            if arr[a] <= n:
                #assert(arr[a] <= n)
                return (arr[a],a)
            
        mid = (a+b) // 2

        if arr[mid]>=n:
            return _bin_search_(a,mid)
        else:
            return _bin_search_(mid,b)
            
    return _bin_search_(0,len(arr)-1)

arr : List[int] = [1,5,10,15]
n   = 0
#print(_inf_search(arr,n))
test_inf(_inf_search)


# Tarea: Valor Central de dos arreglos

Dados dos arreglos ordenados `arr1` y `arr2`, obtenga el valor central del arreglo `(arr1+arr2).sorted` sin unir los arreglos.


```
Input:  arr1 = [1,5,20,25,26], 
        arr2 = [0,3,8,12,23,24,30]
Output: 20
Output: puesto que el arreglo ordenado 
        arr1+arr2 = [0, 1, 3, 5, 8, 12, 20, 23, 24, 25, 26, 30]
        posee 12 posiciones y 20 esta en la 6ta.


```


Hint: Usar una variacion de `Binary Search`

In [ ]:
# Solucion
#  O(N+M)
# arr3 [0,1,3,5]
# la solucion tiene 5 elementos


# La mediana tiene n//2 elementos a su derecha en el arreglo arr1+arr2
# a = 0
# b = 30
# mid = 15

#5
#2
# mid tiene 6


# a = 15
# b = 30
# mid = 22
# mid tiene 5 elementos

def central(xs : List[int], ys: List[int]) -> int:
    

    return 0




# Ejercicio: Merge sort

El algoritmo merge sort es quizas el ejemplo clasico de divida y conquista, funciona de la siguiente manera:


## Divide: 
1. Todo arreglo con 0 o 1 elementos esta ordenado
2. Dado un arreglo `arr` de tamaño `n`, aplicamos `conquer` al resultado de hacer divide a cada mitad del arreglo

## Conquer:
1. Dado dos arreglos `arr_l` y `arr_r` ordenados, los combinamos en tiempo lineal para obtener un arreglo `arr` ordenado.



In [ ]:
def merge_sort(arr : List[int]) -> None:


    def combine(l : List[int], r : List[int]) -> List[int]:
        pass


    def divide(a : int, b:int) -> None:
        pass

def is_sorted_test(sort : Callable[[List[int]], None]) -> None:
    '''
    Dada una funcion para sortear un arreglo in place, determina si es correcta
    verificando el invariante:

    
     
    '''
    @given(ys = st.lists(st.integers()))
    def _is_sorted_test(ys : Any):
        from copy import deepcopy
        xs : List[int] = list(ys)
        zs : List[int] = deepcopy(xs)
        n  : int       = len(xs)
        sort(xs)

        
        # Arreglo esta sorteado si y solo si
        # para todo indice i: 0<= i < n-1
        # xs[i] <= xs[i+1]
        for i in range(n-1):
            try:
                assert(xs[i] <= xs[i+1])
            except Exception as e:
                display_( color_text("Test Failed!",color=ERROR_COLOR) + "<br><br>" +
                    f"El elemento en el indice ${i}$: ${xs[i]}$<br>" + 
                    f"es mayor que el elementl en el indice ${i+1}$: ${xs[i+1]}$ <br>" + 
                    f"arr: {zs} <br>" + 
                    f"after: {xs}"
                    
                    )
                return
        
        #display_(color_text("Test Success!",color=SUCCESS_COLOR))

    _is_sorted_test()

is_sorted_test(merge_sort)
    


